In [56]:
# Import libraries that are required to run your project
# You are allowed to add more libraries as you need

import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pyBigWig
import glob
from tqdm import tqdm
from datetime import datetime
import random
import os
from IPython.display import clear_output


## Work Package 1.1 - Modeling Choices & Data Pre-processing

In [46]:
SIGNALS_CNN = ['DNase', 'H3K4me1', 'H3K4me3', 'H3K27ac', 'H3K36me3']   # List of signal tracks to be used
N_SIGNALS_CNN = len(SIGNALS_CNN)
SIGNAL_CNN_WINDOW = 1e4 # Window size in base pairs
N_BINS = 100 # Number of base pairs per bin
PREPROCESSED_BASE_PATH = "Data/preprocessed"

In [47]:
half_window = int(SIGNAL_CNN_WINDOW // 2)
chromstrs = []
ranges = []


def get_signals_bins(df: pd.DataFrame, cell_type: str, n_bins: int=N_BINS) -> np.ndarray:
    df["neg_strand"] = mask = df['strand'] == '-'
    df.loc[mask, ['TSS_start', 'TSS_end']] = df.loc[mask, ['TSS_end', 'TSS_start']].values
    df["center"] = ((df["TSS_start"] + df["TSS_end"]) // 2).astype(int)
    df["window_start"] = df["center"] - half_window
    df["window_end"] = df["center"] + half_window

    bins_signal_gene = np.zeros((len(df), len(SIGNALS_CNN), N_BINS))
    for i, signal in enumerate(SIGNALS_CNN):
        print("Processing signal:", signal, f"{i+1}/{len(SIGNALS_CNN)}")

        for j, (chromstr, window_start, window_end, neg_strand) in enumerate(tqdm(df[["chr", "window_start", "window_end", "neg_strand"]].itertuples(index=False), total=len(df))):
            fname = glob.glob(f"Data/bigwig/{signal}-bigwig/{cell_type}*")[0]
            bw = pyBigWig.open(fname)

            bins = bw.stats(chromstr, window_start, window_end, type="mean", nBins=n_bins)
            if neg_strand:
                bins = bins[::-1]
            bins_signal_gene[j, i] = bins

            bw.close()
            
    return bins_signal_gene

In [ ]:
cell_types_per_set = {
    "train": ["X1", "X2"],
    "val": ["X1", "X2"],
    "test": ["X3"]
}

for train_test_val, cell_types in cell_types_per_set.items():
    print(f"Processing {train_test_val} set.")
    for cell_type in cell_types:
        print(f"Processing cell type: {cell_type}")
        df = pd.read_csv(f'./Data/CAGE-train/{cell_type}_{train_test_val}_info.tsv', sep='\t', usecols=[0,1,4,5,6])
        signal_bins = get_signals_bins(df, cell_type, n_bins=N_BINS)
        np.save(f'{PREPROCESSED_BASE_PATH}/cnn_input_{cell_type}_{train_test_val}_{N_BINS}.npy', signal_bins)

Group the data for training:

In [49]:
for train_test_val, cell_types in cell_types_per_set.items():
    all_data = []
    for cell_type in cell_types:
        data = np.load(f'{PREPROCESSED_BASE_PATH}/cnn_input_{cell_type}_{train_test_val}_{CNN_N_BINS}.npy')
        all_data.append(data)
    grouped = np.concatenate(all_data, axis=0)
    np.save(f"{PREPROCESSED_BASE_PATH}/cnn_input_{train_test_val}.npy", grouped)    

Do the same with the outputs:

In [ ]:
for train_test_val, cell_types in cell_types_per_set.items():
    if train_test_val == "test":
        continue
    all_y = []
    for cell_type in cell_types:
        y_df = pd.read_csv(f"Data/CAGE-train/{cell_type}_{train_test_val}_y.tsv", sep="\t")
        all_y.append(y_df["gex"].to_numpy())
    np.save(f"{PREPROCESSED_BASE_PATH}/cnn_y_{train_test_val}.npy", np.concatenate(all_y, axis=0))       

## Work Package 1.2 - Model Building

### Define the model

The model used to estimate the gene expression from binned signal is the following:

In [ ]:
class GeneCNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(N_SIGNALS_CNN, 32, kernel_size=3, padding="same"),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(32, 16, kernel_size=3, padding="same"),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(16, 1, kernel_size=3, padding="same"),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.LazyLinear(1),
        )

    def forward(self, x):
        # x shape: (batch, features=2, genes)
        out = self.net(x)
        return out.squeeze(1)  # shape: (batch, genes)

### Train the model
Since we are scoring with Pearson's R, by which only the rank of the predicted elements is considered, we train our model to predict the $\log(y + 1)$. This doesn't affect the score, since $\log$ is monotonically increasing, and will help the training, given that some very high values in $y$ are difficult to model.

In [50]:
class BigWigDataset(Dataset):
    def __init__(self, X, y):
        self.missing_val = np.any(np.isnan(X) | np.isinf(X), axis=(1,2))
        X = X[~self.missing_val, :, :]
        y = y[~self.missing_val]
        self.X = torch.from_numpy(X).float()
        self.y_orig = torch.from_numpy(y).float()
        self.y = torch.log1p(self.y_orig)  # log(y + 1) transformation
    
    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [51]:
BATCH_SIZE = 2056
EPOCHS = 200
LEARNING_RATE = 0.001

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [52]:
x_train = np.load(f"{PREPROCESSED_BASE_PATH}/cnn_input_train.npy")
y_train = np.load(f"{PREPROCESSED_BASE_PATH}/cnn_y_train.npy")

x_val = np.load(f"{PREPROCESSED_BASE_PATH}/cnn_input_val.npy")
y_val = np.load(f"{PREPROCESSED_BASE_PATH}/cnn_y_val.npy")

training_loader = DataLoader(
    BigWigDataset(x_train, y_train),
    batch_size=BATCH_SIZE,
    shuffle=True
)

validation_loader = DataLoader(
    BigWigDataset(x_val, y_val),
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [60]:
net = GeneCNNModel()
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)
best_val_loss = float('inf')
model_path = None

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
for epoch in range(EPOCHS):
    best_loss = float("inf")
    last_loss = 0

    net.train()

    running_loss = .0
    print(f"Epoch: {epoch + 1}/{EPOCHS}")
    
    for i, (X, y) in tqdm(enumerate(training_loader), total=len(training_loader)):            
        optimizer.zero_grad()
        y_hat = net(X)
        loss = loss_fn(y_hat.flatten(), y)
        
        loss.backward()

        optimizer.step()
        batch_loss = loss.item()
        running_loss += batch_loss
        best_loss = min(batch_loss, best_loss)
    
    

    avg_loss = running_loss / (i + 1)

    print(f"Training loss: best {best_loss} and avg {avg_loss}")

    net.eval()

    running_vloss = .0

    with torch.no_grad():
        for i, (X, y) in tqdm(enumerate(validation_loader), total=len(validation_loader)):
            y_hat = net(X)
            loss = loss_fn(y_hat.flatten(), y)
            running_vloss += loss.item()
    
    avg_val_loss = running_vloss / (i + 1)
    print('Avg valid loss: {}'.format(avg_val_loss))

    if avg_val_loss < best_val_loss:
        if model_path and os.path.exists(model_path):
            os.remove(model_path)

        best_val_loss = avg_val_loss

        model_path = "models/cnn_{}_VLOSS{:.4f}_BS{}_LR{}_BIN{}_W{}".format(timestamp, best_val_loss, BATCH_SIZE, LEARNING_RATE, int(N_BINS), int(SIGNAL_CNN_WINDOW))

        print(f"Saving as best model in {model_path}")
        torch.save(net, model_path)

    if (epoch + 1) % 10 == 0:
        clear_output(wait=True)

print("Training complete. Best avg validation loss:", best_val_loss)

Epoch: 21/200


100%|██████████| 14/14 [00:02<00:00,  5.72it/s]


Training loss: best 1.4061312675476074 and avg 1.563666548047747


100%|██████████| 2/2 [00:00<00:00, 15.85it/s]


Avg valid loss: 1.478577733039856
Saving as best model in models/cnn_20251021_014810_VLOSS1.4786_BS2056_LR0.001_BIN100_W10000
Epoch: 22/200


...

Training loss: best 1.3330895900726318 and avg 1.4856048652103968


100%|██████████| 2/2 [00:00<00:00, 14.27it/s]


Avg valid loss: 1.49180006980896
Epoch: 27/200


 71%|███████▏  | 10/14 [00:01<00:00,  5.58it/s]


KeyboardInterrupt: 

## Work Package 1.3 - Prediction on Test Data (Evaluation Metric)

In [ ]:
# ---------------------------INSERT CODE HERE---------------------------




# ----------------------------------------------------------------------

# Check if "pred" meets the specified constrains
assert isinstance(pred, np.ndarray), 'Prediction array must be a numpy array'
assert np.issubdtype(pred.dtype, np.number), 'Prediction array must be numeric'
assert pred.shape[0] == len(test_genes), 'Each gene should have a unique predicted expression'

In [ ]:
model.eval()  # Set the model to evaluation mode
preds = []
for X_chr, y_chr in zip(train_x, train_y):
    # Convert numpy arrays to torch tensors
    X_chr = torch.tensor(X_chr.T, dtype=torch.float32).unsqueeze(0)  # (1, 2, num_genes)
    y_chr = torch.tensor(y_chr, dtype=torch.float32).unsqueeze(0)    # (1, num_genes)

    preds.append(model(X_chr))


In [ ]:
# Convert to numpy if tensor
pred_values = preds[0].detach().cpu().numpy().ravel()  # assuming same chromosome index
gene_names_chr = train_names[0]  # assuming same chromosome index

# Build DataFrame
df_pred = pd.DataFrame({
    'gene_name': gene_names_chr,
    'prediction': pred_values
})

print(df_pred)

#### Store Predictions in the Required Format

In [ ]:
# Store predictions in a ZIP. 
# Upload this zip on the project website under "Your submission".
# Zip this notebook along with the conda environment (and README, optional) and upload this under "Your code".

save_dir = 'Data/submission'  # TODO
file_name = 'gex_predicted.csv'         # PLEASE DO NOT CHANGE THIS
zip_name = "Laffranchi_Paolo_Project1.zip" # TODO
save_path = f'{save_dir}/{zip_name}'
compression_options = dict(method="zip", archive_name=file_name)

test_genes = pd.read_csv(f"{save_dir}/{file_name}")
test_genes[['gene_name', 'gex_predicted']].to_csv(save_path, compression=compression_options)